In [34]:
import glob
import os

import dask.dataframe as dd
import pandas as pd
from dask.diagnostics import ProgressBar
from sc2.bot_ai import Race

In [35]:
maps = [
    "AutomatonAIE.SC2Map",
    "AbyssalReefAIE.SC2Map",
    "InterloperAIE.SC2Map",
    "EphemeronAIE.SC2Map",
    "AcropolisAIE.SC2Map",
    "ThunderbirdAIE.SC2Map",
]
test_maps = {maps[0]}
races = list(map(str, Race))

In [36]:
replay_dir = "../resources/replays/aiarena/*.SC2Replay.parquet"
replay_paths = glob.glob(replay_dir)[:-1]
replay_names = [os.path.basename(p) for p in replay_paths]
replay_paths

['../resources/replays/aiarena\\3901860_Sharkling_PhantomBot_EphemeronAIE.SC2Replay.parquet',
 '../resources/replays/aiarena\\3904878_Deimos_Eris_AutomatonAIE.SC2Replay.parquet',
 '../resources/replays/aiarena\\3904909_VeTerran-revived_Eris_AbyssalReefAIE.SC2Replay.parquet',
 '../resources/replays/aiarena\\3904922_Ketroc_Eris_InterloperAIE.SC2Replay.parquet',
 '../resources/replays/aiarena\\3905061_MicroMachine_Eris_InterloperAIE.SC2Replay.parquet',
 '../resources/replays/aiarena\\3905138_Eris_Aeolus_AbyssalReefAIE.SC2Replay.parquet',
 '../resources/replays/aiarena\\3905139_Eris_Xena_EphemeronAIE.SC2Replay.parquet',
 '../resources/replays/aiarena\\3905140_Eris_Zozo_AcropolisAIE.SC2Replay.parquet',
 '../resources/replays/aiarena\\3905142_Eris_Caninana_AbyssalReefAIE.SC2Replay.parquet',
 '../resources/replays/aiarena\\3905145_Eris_Phobos_AcropolisAIE.SC2Replay.parquet',
 '../resources/replays/aiarena\\3905146_Eris_SharpenedEdge_AutomatonAIE.SC2Replay.parquet',
 '../resources/replays/aiar

In [37]:
pd.read_parquet(replay_paths[0])

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [27]:
def pipeline(replay_path: str) -> pd.DataFrame:
    replay_name = os.path.basename(replay_path)
    df = pd.read_parquet(replay_path)
    df = df.groupby(["game_loop", "unit_type", "player", "owner"])["tag"]
    df = df.count()
    df = df.rename("count")
    df = df.reset_index()
    df["replay_name"] = replay_name
    return df

In [28]:
with ProgressBar():
    df = dd.from_map(pipeline, replay_paths).compute()

[########################################] | 100% Completed | 1.02 ss
[########################################] | 100% Completed | 209.09 s


In [29]:
test_mask = df["game_loop"] % 16 == 0
df[test_mask].reset_index().to_parquet("../resources/datasets/scout/test.parquet")
df[~test_mask].reset_index().to_parquet("../resources/datasets/scout/train.parquet")